In [21]:
import numpy as np
import math

muN = -1.91  # Neutron magnetic moment
Pn = 0.95  # Neutron polarization factor
m = 0.938272  # Nucleon mass

def weighted_avg(values, errors):
    weights = 1 / errors**2
    return np.sum(values * weights) / np.sum(weights), np.sqrt(1 / np.sum(weights))

def combine_arrays(arrays):
    return np.concatenate(arrays)

def combine_scalars_weighted(vals, errors):
    weights = 1 / np.array(errors)**2
    return np.sum(vals * weights) / np.sum(weights)

def combine_errors(vals, errors):
    weights = 1 / np.array(errors)**2
    return np.sqrt(np.sum((errors * weights)**2)) / np.sum(weights)

# Load data
kin4a = np.load("kin4saves/kin4a.npz")
kin4b = np.load("kin4saves/kin4b.npz")

A_phys_all = combine_arrays([kin4a["A_phys_per_run"], kin4b["A_phys_per_run"]])
A_phys_stat_all = combine_arrays([kin4a["statistical_error_per_run"], kin4b["statistical_error_per_run"]])
P_beam_all = combine_arrays([kin4a["P_beam_per_run"], kin4b["P_beam_per_run"]])
P_target_all = combine_arrays([kin4a["P_target_per_run"], kin4b["P_target_per_run"]])
P_beamE_all = combine_arrays([kin4a["P_beamE_per_run"], kin4b["P_beamE_per_run"]])
P_targetE_all = combine_arrays([kin4a["P_targetE_per_run"], kin4b["P_targetE_per_run"]])

# Raw asymmetry and counts
A_raw_all = combine_arrays([kin4a["A_raw_per_run"], kin4b["A_raw_per_run"]])
A_raw_error_all = combine_arrays([kin4a["A_raw_error_per_run"], kin4b["A_raw_error_per_run"]])
N_plus_raw = kin4a["N_plus_raw"] + kin4b["N_plus_raw"]
N_minus_raw = kin4a["N_minus_raw"] + kin4b["N_minus_raw"]
N_total = kin4a["N_total"] + kin4b["N_total"]
N_p = kin4a["N_p"] + kin4b["N_p"]
N_n = kin4a["N_n"] + kin4b["N_n"]
N_bg = kin4a["N_bg"] + kin4b["N_bg"]

mask = A_phys_stat_all != 0

A_phys_all = A_phys_all[mask]
A_phys_stat_all = A_phys_stat_all[mask]
P_beam_all = P_beam_all[mask]
P_target_all = P_target_all[mask]
P_beamE_all = P_beamE_all[mask]
P_targetE_all = P_targetE_all[mask]
A_raw_all = A_raw_all[mask]
A_raw_error_all = A_raw_error_all[mask]

A_Phys_wavg, A_Phys_wavgE = weighted_avg(A_phys_all, A_phys_stat_all)
A_raw, A_raw_error = weighted_avg(A_raw_all, A_raw_error_all)
P_beam_avg, P_beam_error_avg = weighted_avg(P_beam_all, A_phys_stat_all)
P_target_avg, P_target_error_avg = weighted_avg(P_target_all, A_phys_stat_all)

def combine_scalar_param(name, use_errors=True):
    if use_errors:
        try:
            values = [kin4a[name], kin4b[name]]
            errors = [kin4a[name + "_error"], kin4b[name + "_error"]]
            return combine_scalars_weighted(values, errors)
        except KeyError:
            pass
    return 0.5 * (kin4a[name] + kin4b[name])

f_acc = combine_scalar_param("f_acc")
f_p = combine_scalar_param("f_p")
f_bg = combine_scalar_param("f_bg")
f_n = combine_scalar_param("f_n")
f_nitrogen = combine_scalar_param("f_nitrogen")
f_inelastic = f_bg - f_acc - f_nitrogen

A_acc = combine_scalar_param("A_acc")
A_p = combine_scalar_param("A_p")
A_inelastic = combine_scalar_param("A_inelastic")

f_p_error = combine_errors([kin4a["f_p"], kin4b["f_p"]], [kin4a["f_p_error"], kin4b["f_p_error"]])
f_bg_error = combine_errors([kin4a["f_bg"], kin4b["f_bg"]], [kin4a["f_bg_error"], kin4b["f_bg_error"]])
f_n_error = combine_errors([kin4a["f_n"], kin4b["f_n"]], [kin4a["f_n_error"], kin4b["f_n_error"]])
f_nitrogen_error = combine_errors([kin4a["f_nitrogen"], kin4b["f_nitrogen"]], [kin4a["f_nitrogen_error"], kin4b["f_nitrogen_error"]])
f_acc_error = combine_errors([kin4a["f_acc"], kin4b["f_acc"]], [kin4a["f_acc_error"], kin4b["f_acc_error"]])

A_acc_error = combine_errors([kin4a["A_acc"]], [kin4a["A_acc_error"]])
A_p_sig = combine_errors([kin4a["A_p"]], [kin4a["A_p_error"]])
A_inelastic_error = combine_errors([kin4a["A_inelastic"]], [kin4a["A_inelastic_error"]])

sig_Pb = P_beam_error_avg
sig_Pt = P_target_error_avg
P_total = P_beam_avg * P_target_avg * Pn
sig_P_total = P_total * ((sig_Pb / P_beam_avg)**2 + (sig_Pt / P_target_avg)**2)**0.5

fn = 1 - f_acc - f_p - f_inelastic - f_nitrogen
A = A_Phys_wavg
P = P_total

dAdf1 = ((A * P - A_acc) / (P * fn) * f_acc_error)**2
dAdf2 = ((A * P - A_p) / (P * fn) * f_p_error)**2
dAdf3 = ((A * P - A_inelastic) / (P * fn) * f_bg_error)**2
dAdf4 = ((A * P) / (P * fn) * f_nitrogen_error)**2
dAdA1 = (f_acc / (P * fn) * A_acc_error)**2
dAdA2 = (f_p / (P * fn) * A_p_sig)**2
dAdA3 = (f_inelastic / (P * fn) * A_inelastic_error)**2
dAdP = (A * sig_P_total)**2

A_phys_sys_error = np.sqrt(dAdf1 + dAdf2 + dAdf3 + dAdf4 + dAdA1 + dAdA2 + dAdA3 + dAdP)

Q2_avg = combine_scalar_param("Q2_avg")
tau_avg = Q2_avg / (4 * m**2)
epsilon_avg = 1 / (1 + 2 * (1 + tau_avg) * math.tan(kin4a["etheta_avg"] / 2)**2)

Px_avg = combine_scalar_param("Px_avg")
Pz_avg = combine_scalar_param("Pz_avg")

A_term = epsilon_avg / tau_avg * A_Phys_wavg
B_term = np.sqrt((2 * epsilon_avg) * (1 - epsilon_avg) / tau_avg) * Px_avg
C_term = A_Phys_wavg + np.sqrt(1 - epsilon_avg**2) * Pz_avg

A_error_stat = (epsilon_avg / tau_avg) * A_Phys_wavgE
C_error_stat = A_Phys_wavgE
A_error_sys = (epsilon_avg / tau_avg) * A_phys_sys_error
C_error_sys = A_phys_sys_error

discriminant = B_term**2 - 4 * A_term * C_term
ratio = (-B_term + np.sqrt(discriminant)) / (2 * A_term)
ratio_error_stat = np.sqrt(((C_term / (A_term * np.sqrt(discriminant)) + ratio / A_term)**2) * A_error_stat**2 + (C_error_stat**2 / discriminant))
ratio_error_sys = np.sqrt(((C_term / (A_term * np.sqrt(discriminant)) + ratio / A_term)**2) * A_error_sys**2 + (C_error_sys**2 / discriminant))

statdict = {
    "N_p": N_p,
    "N_n": N_n,
    "N_bg": N_bg,
    "N_total": N_total,
    "f_p": f_p,
    "f_p_e": f_p_error,
    "f_bg": f_bg,
    "f_bg_e": f_bg_error,
    "f_n": f_n,
    "f_n_e": f_n_error,
    "f_nitrogen": f_nitrogen,
    "f_nitrogen_e": f_nitrogen_error,
    "f_acc": f_acc,
    "f_acc_e": f_acc_error,
    "A_acc": A_acc,
    "A_acc_e": A_acc_error,
    "A_p": A_p,
    "A_p_e": A_p_sig,
    "A_inelastic": A_inelastic,
    "A_inelastic_e": A_inelastic_error,
    "P_beam": P_beam_avg,
    "P_beam_e": P_beam_error_avg,
    "P_target": P_target_avg,
    "P_target_e": P_target_error_avg,
    "N_plus_raw": N_plus_raw,
    "N_minus_raw": N_minus_raw,
    "A_raw": A_raw,
    "A_raw_rel_error": A_raw_error / A_raw,
    "A_Phys_wavg": A_Phys_wavg,
    "A_phys_rel_sys_error": A_phys_sys_error / A_Phys_wavg,
    "A_phys_rel_sat_error": A_Phys_wavgE / A_Phys_wavg,
    "tau": tau_avg,
    "epsilon": epsilon_avg,
    "Q2": Q2_avg,
    "Px": Px_avg,
    "Pz": Pz_avg,
    "GE/GM * muN": ratio * muN,
    "GE/GM_stat_error": abs(ratio_error_stat * muN),
    "GE/GM_sys_error": abs(ratio_error_sys * muN)
}

import pandas as pd

# Display as table
df = pd.DataFrame([statdict])
print(df.T)  # Transpose for vertical view


                                 0
N_p                     535.570872
N_n                    3988.979501
N_bg                  12962.433780
N_total               17795.000000
f_p                       0.030094
f_p_e                     0.001320
f_bg                      0.695824
f_bg_e                    0.008200
f_n                       0.186124
f_n_e                     0.003567
f_nitrogen                0.019068
f_nitrogen_e              0.000270
f_acc                     0.041002
f_acc_e                   0.001561
A_acc                     0.025422
A_acc_e                   0.019245
A_p                      -0.000164
A_p_e                     0.000070
A_inelastic               0.003550
A_inelastic_e             0.001098
P_beam                    0.834979
P_beam_e                  0.071001
P_target                  0.516489
P_target_e                0.071001
N_plus_raw             8513.000000
N_minus_raw            8234.000000
A_raw                     0.020419
A_raw_rel_error     

In [22]:
with open("kin4combinedresults.txt",'w') as f:
    for i,key in enumerate(statdict):
        if i<4:
            f.write(key+","+f"{statdict[key]:.0f}\n")
        else:
            f.write(key+","+f"{statdict[key]:.4f}\n")